In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp template.config

In [ ]:
# export
from typing import List
from fastcore.script import call_parse, Param
import inspect
import os
import importlib
import json
from pathlib import Path

from pymemri.plugin.pluginbase import get_plugin_cls
from pymemri.pod.client import PodClient

In [ ]:
# hide
# test imports
from pymemri.plugin.pluginbase import PluginBase
from pprint import pprint
import os

# Template Configuration formatter

To do plugin configuration in the front-end, plugins require a `config.json` in the root of the plugin repository. This module contains utilities to directly generate this config file from the plugin definition

Example usage:
```
create_plugin_config
```

In [ ]:
# export
ALLOWED_TYPES = [int, str, float]

In [ ]:
# export
def get_params(cls):
    params = inspect.signature(cls.__init__).parameters
    return {k: v for k, v in list(params.items()) if k not in {"self", "args", "kwargs"}}

def identifier_to_displayname(identifier: str) -> str:
    return identifier.replace("_", " ").title()

def get_param_config(name, dtype, is_optional, default):
    return {
        "name": name,
        "display": identifier_to_displayname(name),
        "data_type": dtype,
        "type": "textbox",
        "default": default,
        "optional": is_optional,
    }

def create_config(plugin_cls: type) -> List[dict]:
    config = list()
    for param_name, param in get_params(plugin_cls).items():
        if param_name.startswith("_"):
            continue
        if param.annotation == inspect._empty:
            print(f"Skipping unannotated parameter `{param_name}`")
            continue
        if param.annotation not in ALLOWED_TYPES:
            print(f"Skipping parameter with unknown type: `{param_name}: {param.annotation}`")
            continue
        is_optional = param.default != inspect._empty
        dtype = PodClient.TYPE_TO_SCHEMA[param.annotation]
        default = param.default if is_optional else None
        param_config = get_param_config(param_name, dtype, is_optional, default)
        config.append(param_config)
    return config

In [ ]:
# export
@call_parse
def create_plugin_config(
    metadata: Param("metadata.json of the plugin", str) = None,
    tgt_file: Param("Filename of config file", str) = "config.json"):
    if metadata is None:
        if os.path.exists("./metadata.json"):
            metadata = "./metadata.json"
        else:
            print("Define a metadata file with --metadata <filename>")
            return
    with open(metadata, "r") as f:
        metadata = json.load(f)
        
    plugin_module = metadata["pluginModule"]
    plugin_name = metadata["pluginName"]
    
    try:
        plugin_cls = get_plugin_cls(plugin_module, plugin_name)
    except Exception as e:
        print(e)
        return
    config = create_config(plugin_cls)
    
    with open(tgt_file, "w") as f:
        json.dump(config, f, indent=2)
    print(f"Config saved to {Path(tgt_file)}")

In [ ]:
class MyPlugin(PluginBase):
    def __init__(self, my_arg: str, my_str: str = None, my_int: int = None, my_float: float = None, unannotated=None, _private = None, **kwargs):
        super().__init__(**kwargs)
    
    def run(self):
        pass
    
    def add_to_schema(self):
        pass

In [ ]:
config = create_config(MyPlugin)
pprint(config)

Skipping unannotated parameter `unannotated`
[{'data_type': 'Text',
  'default': None,
  'display': 'My Arg',
  'name': 'my_arg',
  'optional': False,
  'type': 'textbox'},
 {'data_type': 'Text',
  'default': None,
  'display': 'My Str',
  'name': 'my_str',
  'optional': True,
  'type': 'textbox'},
 {'data_type': 'Integer',
  'default': None,
  'display': 'My Int',
  'name': 'my_int',
  'optional': True,
  'type': 'textbox'},
 {'data_type': 'Real',
  'default': None,
  'display': 'My Float',
  'name': 'my_float',
  'optional': True,
  'type': 'textbox'}]


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.api.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted template.config.ipynb.
Converted template.formatter.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
